In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import auc,roc_auc_score,confusion_matrix

In [2]:
data = pd.read_csv("Dev_Sample1.csv")
A = data.iloc[:,1:282]
C=pd.DataFrame(A)
c=C.fillna(C.mean())
X=pd.DataFrame(c)

In [3]:
y = data['y']

In [4]:
data5 = pd.read_csv("OOT_Sample.csv")
A5 = data5.iloc[:,1:282]
C5=A5.copy()
c5=C5.fillna(C5.mean())
X5=c5.copy()

In [5]:
y5 = data5["y"] 

In [6]:
model = RandomForestClassifier(random_state=123)
model.fit(X, y) 

C:\Users\IN16\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

In [7]:
y_prob=model.predict_proba(X)   

In [15]:
ks = pd.DataFrame()
ks['score'] = y_prob[:,1]
ks['actual'] = data['y'].reset_index(drop = True)
ks = ks.sort_values(by = 'score' , ascending = False).reset_index(drop=True)
ks['decile'] = pd.cut(ks.index,10, labels= range(10))
ks['bad'] = ks['actual']
ks['goods'] = 1 - ks['bad']

agg1 = pd.DataFrame()
agg1['min_scr'] = ks.groupby(by = 'decile').score.min()
agg1['max_scr'] = ks.groupby(by = 'decile').score.max()
agg1['bad'] = ks.groupby(by = 'decile').bad.sum()
agg1['goods'] = ks.groupby(by = 'decile').goods.sum()
agg1['total'] = agg1['bad'] + agg1['goods']
agg1['bad_rate'] = agg1['bad']/(agg1['bad'].sum())
agg1['cumbad'] = agg1['bad_rate'].cumsum()
agg1['good_rate'] = agg1['goods']/(agg1['goods'].sum())
agg1['cumgoods'] = agg1['good_rate'].cumsum()
agg1["ks"] = agg1['cumbad'] - agg1['cumgoods']
agg1

,min_scr,max_scr,bad,goods,total,bad_rate,cumbad,good_rate,cumgoods,ks
decile,,,,,,,,,,
0,0.3,0.9,1725,27645,29370,0.099361,0.099361,0.100041,0.100041,-0.000680
1,0.2,0.3,1737,27633,29370,0.100052,0.199412,0.099997,0.200038,-0.000625
2,0.2,0.2,1724,27646,29370,0.099303,0.298716,0.100044,0.300082,-0.001366
3,0.1,0.2,1788,27582,29370,0.102989,0.401705,0.099813,0.399894,0.001811
4,0.1,0.1,1778,27592,29370,0.102413,0.504118,0.099849,0.499743,0.004375
5,0.1,0.1,1689,27680,29369,0.097287,0.601405,0.100167,0.599910,0.001495
6,0.0,0.1,1737,27633,29370,0.100052,0.701457,0.099997,0.699907,0.001550
7,0.0,0.0,1723,27647,29370,0.099245,0.800703,0.100048,0.799955,0.000748
8,0.0,0.0,1695,27675,29370,0.097633,0.898335,0.100149,0.900104,-0.001769


In [9]:
ks_score=max(agg1["ks"])
ks_score*100

95.63422999766284

In [10]:
y_prob=model.predict_proba(X5[X.columns])

In [11]:
ks = pd.DataFrame()
ks['score'] = y_prob[:,1]
ks['actual'] = data5['y'].reset_index(drop = True)
ks = ks.sort_values(by = 'score' , ascending = False).reset_index(drop=True)
ks['decile'] = pd.cut(ks.index,10, labels= range(10))
ks['bad'] = ks['actual']
ks['goods'] = 1 - ks['bad']

agg1 = pd.DataFrame()
agg1['min_scr'] = ks.groupby(by = 'decile').score.min()
agg1['max_scr'] = ks.groupby(by = 'decile').score.max()
agg1['bad'] = ks.groupby(by = 'decile').bad.sum()
agg1['goods'] = ks.groupby(by = 'decile').goods.sum()
agg1['total'] = agg1['bad'] + agg1['goods']
agg1['bad_rate'] = agg1['bad']/(agg1['bad'].sum())
agg1['cumbad'] = agg1['bad_rate'].cumsum()
agg1['good_rate'] = agg1['goods']/(agg1['goods'].sum())
agg1['cumgoods'] = agg1['good_rate'].cumsum()
agg1["ks"] = agg1['cumbad'] - agg1['cumgoods']
agg1


,min_scr,max_scr,bad,goods,total,bad_rate,cumbad,good_rate,cumgoods,ks
decile,,,,,,,,,,
0,0.3,0.9,3814,25556,29370,0.206497,0.206497,0.092854,0.092854,1.136434e-01
1,0.2,0.3,2457,26913,29370,0.133027,0.339524,0.097784,0.190638,1.488859e-01
2,0.2,0.2,2333,27037,29370,0.126313,0.465836,0.098235,0.288872,1.769643e-01
3,0.1,0.2,1765,27605,29370,0.095560,0.561397,0.100298,0.389170,1.722264e-01
4,0.1,0.1,1619,27751,29370,0.087656,0.649053,0.100829,0.489999,1.590533e-01
5,0.1,0.1,1690,27679,29369,0.091500,0.740552,0.100567,0.590566,1.499858e-01
6,0.0,0.1,1405,27965,29370,0.076069,0.816622,0.101606,0.692173,1.244488e-01
7,0.0,0.0,1177,28193,29370,0.063725,0.880347,0.102435,0.794607,8.573911e-02
8,0.0,0.0,1128,28242,29370,0.061072,0.941419,0.102613,0.897220,4.419838e-02


In [12]:
ks_score=max(agg1["ks"])
ks_score*100

17.69643160741848